# Pràctica 3 - PLH

### Realitzada pels alumnes Lluc Furriols i Pau Prat Moreno

In [20]:
'''
import os 
f = open("/dev/null", "w")
os.dup2(f.fileno(), 2)
f.close()

import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()
'''

'\nimport os \nf = open("/dev/null", "w")\nos.dup2(f.fileno(), 2)\nf.close()\n\nimport nltk\nimport ssl\n\ntry:\n    _create_unverified_https_context = ssl._create_unverified_context\nexcept AttributeError:\n    pass\nelse:\n    ssl._create_default_https_context = _create_unverified_https_context\n\nnltk.download()\n'

In [21]:
import nltk
nltk.download('punkt', quiet=True) # Tokenitzador
nltk.download('averaged_perceptron_tagger', quiet=True) # Etiquetador POS
nltk.download('maxent_ne_chunker', quiet=True) # Etiquetador Entitats Anomenades
nltk.download('words', quiet=True)

True

In [22]:
# Fet pel profe a la pissarra
'''
class FeatureGetter:
    def __init__(ws_cap, ws_prefix,...):
        self.ws_cap = ws_cap
        self.ws_prefix = ws_prefix
        ...
    def __call__(self, token, idx):
        pass
getter_1 = FeatureGetter(ws_cap=True, ws_prefix=False, ...)
'''

'\nclass FeatureGetter:\n    def __init__(ws_cap, ws_prefix,...):\n        self.ws_cap = ws_cap\n        self.ws_prefix = ws_prefix\n        ...\n    def __call__(self, token, idx):\n        pass\ngetter_1 = FeatureGetter(ws_cap=True, ws_prefix=False, ...)\n'

In [23]:
nltk.download('conll2002')
from nltk.corpus import conll2002

train_esp = conll2002.iob_sents('esp.train') # Train, 
val_esp = conll2002.iob_sents('esp.testa') # Val
test_esp = conll2002.iob_sents('esp.testb') # Test

train_ned = conll2002.iob_sents('ned.train') # Train
val_ned = conll2002.iob_sents('ned.testa') # Val
test_ned = conll2002.iob_sents('ned.testb') # Test

# Convertir token postag classe --> postag classe (crec, ha dit 2)

[nltk_data] Downloading package conll2002 to
[nltk_data]     C:\Users\llucfurriols\AppData\Roaming\nltk_data...
[nltk_data]   Package conll2002 is already up-to-date!


In [24]:
train_esp

[[('Melbourne', 'NP', 'B-LOC'), ('(', 'Fpa', 'O'), ('Australia', 'NP', 'B-LOC'), (')', 'Fpt', 'O'), (',', 'Fc', 'O'), ('25', 'Z', 'O'), ('may', 'NC', 'O'), ('(', 'Fpa', 'O'), ('EFE', 'NC', 'B-ORG'), (')', 'Fpt', 'O'), ('.', 'Fp', 'O')], [('-', 'Fg', 'O')], ...]

In [25]:
"""def feature_func(tokens, idx):
    
    Feature function for CRF NER
    :param tokens: a list of tuples, each tuple containing (word, pos, iob_tag)
    :param idx: the index of the word
    
    # Inicialitzar el diccionari de característiques
    word, pos, iob_tag = tokens[idx]
    features = {
        'word': word,
        'word.lower()': word.lower(),
        'is_first': idx == 0,
        'is_last': idx == len(tokens) - 1,
        'is_capitalized': word[0].upper() == word[0],
        'is_all_caps': word.upper() == word,
        'is_all_lower': word.lower() == word,
        'prefix-1': word[0] if len(word) > 0 else '',
        'prefix-2': word[:2] if len(word) > 1 else '',
        'prefix-3': word[:3] if len(word) > 2 else '',
        'suffix-1': word[-1] if len(word) > 0 else '',
        'suffix-2': word[-2:] if len(word) > 1 else '',
        'suffix-3': word[-3:] if len(word) > 2 else '',
        'prev_word': '' if idx == 0 else tokens[idx - 1][0],
        'next_word': '' if idx == len(tokens) - 1 else tokens[idx + 1][0],
        'has_hyphen': '-' in word,
        'is_numeric': word.isdigit(),
        'pos': pos,
        'pos_prefix-2': pos[:2],
        'pos_prefix-3': pos[:3],
        'iob_tag': iob_tag
    }
    
    # Característiques de la paraula anterior
    if idx > 0:
        prev_word, prev_pos, prev_iob_tag = tokens[idx - 1]
        features.update({
            'prev_word': prev_word,
            'prev_word.lower()': prev_word.lower(),
            'prev_word.is_capitalized': prev_word[0].upper() == prev_word[0],
            'prev_pos': prev_pos,
            'prev_iob_tag': prev_iob_tag
        })
    else:
        features['BOS'] = True  # Indicador de començament de sentència
    
    # Característiques de la paraula següent
    if idx < len(tokens) - 1:
        next_word, next_pos, next_iob_tag = tokens[idx + 1]
        features.update({
            'next_word': next_word,
            'next_word.lower()': next_word.lower(),
            'next_word.is_capitalized': next_word[0].upper() == next_word[0],
            'next_pos': next_pos,
            'next_iob_tag': next_iob_tag
        })
    else:
        features['EOS'] = True  # Indicador de final de sentència

    return features
"""

"def feature_func(tokens, idx):\n    \n    Feature function for CRF NER\n    :param tokens: a list of tuples, each tuple containing (word, pos, iob_tag)\n    :param idx: the index of the word\n    \n    # Inicialitzar el diccionari de característiques\n    word, pos, iob_tag = tokens[idx]\n    features = {\n        'word': word,\n        'word.lower()': word.lower(),\n        'is_first': idx == 0,\n        'is_last': idx == len(tokens) - 1,\n        'is_capitalized': word[0].upper() == word[0],\n        'is_all_caps': word.upper() == word,\n        'is_all_lower': word.lower() == word,\n        'prefix-1': word[0] if len(word) > 0 else '',\n        'prefix-2': word[:2] if len(word) > 1 else '',\n        'prefix-3': word[:3] if len(word) > 2 else '',\n        'suffix-1': word[-1] if len(word) > 0 else '',\n        'suffix-2': word[-2:] if len(word) > 1 else '',\n        'suffix-3': word[-3:] if len(word) > 2 else '',\n        'prev_word': '' if idx == 0 else tokens[idx - 1][0],\n       

In [26]:
"""def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

X_train = [sent2features(s) for s in train_esp]
y_train = [sent2labels(s) for s in train_esp]

X_test = [sent2features(s) for s in test_esp]
y_test = [sent2labels(s) for s in test_esp]"""

"def word2features(sent, i):\n    word = sent[i][0]\n    postag = sent[i][1]\n\n    features = {\n        'bias': 1.0,\n        'word.lower()': word.lower(),\n        'word[-3:]': word[-3:],\n        'word.isupper()': word.isupper(),\n        'word.istitle()': word.istitle(),\n        'word.isdigit()': word.isdigit(),\n        'postag': postag,\n        'postag[:2]': postag[:2],\n    }\n    if i > 0:\n        word1 = sent[i-1][0]\n        postag1 = sent[i-1][1]\n        features.update({\n            '-1:word.lower()': word1.lower(),\n            '-1:word.istitle()': word1.istitle(),\n            '-1:word.isupper()': word1.isupper(),\n            '-1:postag': postag1,\n            '-1:postag[:2]': postag1[:2],\n        })\n    else:\n        features['BOS'] = True\n\n    if i < len(sent)-1:\n        word1 = sent[i+1][0]\n        postag1 = sent[i+1][1]\n        features.update({\n            '+1:word.lower()': word1.lower(),\n            '+1:word.istitle()': word1.istitle(),\n          

In [27]:
test_esp

[[('La', 'DA', 'B-LOC'), ('Coruña', 'NC', 'I-LOC'), (',', 'Fc', 'O'), ('23', 'Z', 'O'), ('may', 'NC', 'O'), ('(', 'Fpa', 'O'), ('EFECOM', 'NP', 'B-ORG'), (')', 'Fpt', 'O'), ('.', 'Fp', 'O')], [('-', 'Fg', 'O')], ...]

In [28]:
#%pip install sklearn-pycrfsuite



In [29]:
#%pip install python-crfsuite

In [30]:
from nltk.tag import CRFTagger
import pycrfsuite

ct = CRFTagger()
#quitar la segunda posicion de todos los elementos de la lista train_esp
train_esp = [[(word, iob) for word, pos, iob in sent] for sent in train_esp]
test_esp = [[(word, iob) for word, pos, iob in sent] for sent in test_esp]



ct.train(train_esp, 'model.crf.tagger')
ct.accuracy(test_esp)

Training model... done.


0.9624706498748374

Aquesta accuracy no és real --> Exemple que va posar el profe. Sha de fer servir el recall crec
